# Active Inference Design Agent (demo)

First, let us generate 10-20 audio signals from HA algorithm with different gains and within different contexts.

In [6]:
using Revise
using Rocket
using ReactiveMP
using GraphPPL
using Distributions
using LinearAlgebra
import ProgressMeter
using WAV
using Plots

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1317


In [9]:
# include auxilary functions for data processing
include("helpers/aida_segmentation.jl")
# include SNR quality metric
include("helpers/aida_snr.jl")
# include SNR quality metric
include("helpers/aida_ar.jl")
# include models and corresponding inference algos
include("models_inferences.jl")

batch_coupled_learning (generic function with 1 method)

In [3]:
fs = 8000

8000

In this demo we will work with two different contexts (environemtns): babble and train noises. 

You can think of a user who keeps wandering around a train station: sometimes train arrives and he/she hears the train noises. When there is no train arriving to the platform, the user hears the babble from people waiting for the train.
When someone starts talking to the user he would (maybe) prefer to damp the environmental noise and focus only on the speaker.

Another possible scenario you can think of is the user who steps out of the train and goes to the bar, where people produce babble noise :D 

## Let's obtain priors for the environment

To make our model identifiabile, we would like to obtain priors for the environmental noises. To do this, we use Voice-Activity-Detection (VAD) to find the silent segments (with no speech). When those frames are identified, we fit them to AR model of order 2 to learn the parameters of AR.

Surely, you can obtain the priors based on different logic. (to discuss (1) it doesn't have to be AR, (2) it doesn't have to be VAD)

In [10]:
# Load sound from .wav
babble, fs = wavread("sound/AIDA/training/sp01_babble_sn0.wav")
# split babble into overlapping segments (default 0.01s=10ms, 0.0025=2.5ms)
bbl_seg = get_frames(babble, fs)
# compute number of segments
bbl_totseg = size(bbl_seg, 1)

376

In [11]:
train, fs = wavread("sound/AIDA/training/sp11_train_sn0.wav")
tr_seg = get_frames(train, fs)
tr_totseg = size(tr_seg, 1)

391

In [3]:
# Priors for contexts were obtained through running VAD-AR block
# prior for train noise
trmη = [0.4132104755921993 -0.16961911841949667]
trvη = [0.00972993440498344 -0.0027103005529199036; -0.0027103005529199036 0.004281987640515784]
# trτ  = (41.0, 0.03644943410647206)
trτ  = (41.0, 0.05)

# prior for babble noise
bblmη = [1.1192255902602752 -0.43086292293101314]
bblvη = [0.007837790430663492 -0.005039080815241558; -0.005039080815241558 0.00596413119195013]
# bblτ  = (41.0, 0.0029780512310493387);
bblτ  = (41.0, 0.05);

### VAD block
You can skip this entire block if you used priors from the above cell.

In [7]:
include("models/everything.jl")

lar_inference_ex (generic function with 1 method)

In [9]:
import WAV
include("helpers/aida_segmentation.jl")
include("helpers/aida_ar.jl")

ar_ssm (generic function with 1 method)

In [95]:
# Choose the auido signal from where you intend to extract the parameters
context_seg = bbl_seg
totseg = bbl_totseg
# context_seg = tr_seg
# totseg = tr_totseg

376

In [55]:
ar_order = 10
vmp_iter = 50
fe_ar = zeros(totseg, vmp_iter)
fe_gaussian = zeros(totseg, vmp_iter);

In [56]:
ProgressMeter.@showprogress for segnum in 1:totseg
    inputs, outputs = ar_ssm(context_seg[segnum, :], ar_order)
    γ, τ, θ, x, fe = lar_inference_ex(outputs, ar_order, vmp_iter)
    mθ, vθ = mean(θ), cov(θ)
    mγ = mean(γ)
    fe_ar[segnum, :] = fe
    
    x, γ, fe = inference_gaussian(outputs, vmp_iter, 1e4)
    mx, vx = mean(x), cov(x)
    mγ = mean(γ)
    fe_gaussian[segnum, :] = fe
end

Progress: 100%|█████████████████████████████████████████| Time: 0:08:47


In [57]:
# detect silent frames based on model comparison
vad = [x[end] < y[end] for (x, y) in zip(eachrow(fe_ar), eachrow(fe_gaussian))];

In [58]:
rsegs = findall(isequal(1), vad)
nsegs = findall(isequal(0), vad);

In [59]:
inv(var(sum([context_seg[nseg, :] for nseg in nsegs])))

17.938993996124832

In [60]:
cmx, cvx, cmη, cvη, cτ = lar_batch_learning(hcat([context_seg[nseg, :] for nseg in nsegs]...)', 2, 10, 1e-12);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:06


In [64]:
println("$(mean(cmη, dims=1)), $(mean(cvη, dims=1)[1, :, :]), , $((cτ[end][1], cτ[end][2]))")

[1.1192255902602752 -0.43086292293101314], [0.007837790430663492 -0.005039080815241558; -0.005039080815241558 0.00596413119195013], , (41.0, 0.0029780512310493387)


In [ ]:
# Priors for contexts were obtained through running VAD block
trmη = mean(cmη, dims=1)
trvη = mean(cvη, dims=1)[1, :, :]
trτ  = (cτ[end][1], cτ[end][2])

# prior for babble noise
bblmη = mean(cmη, dims=1)
bblvη = mean(cvη, dims=1)[1, :, :]
bblτ  = (cτ[end][1], cτ[end][2]);

## Source seperation
When priors for the contexts are indentified, we can run noise reduction algorithm based on coupled AR: AR_speech + AR_envrionment = output. This algorithm seperates speech (**z**) and noise (**x**)

We will split our dataset into training and test set. We use signals from training set to learn the mapping function between the gains proposed by agent and aprraisals provided by user. Bare in mind that this split is not necessary and in theory we can get along without it.

In [1]:
# coupled AR model is deisgned to work with time-varying priors for both speech and environmental noise
# prior_to_priors map "static" priors to the corresponding matrices with equal elements
function prior_to_priors(mη, vη, τ, totseg)
    ar_order = size(mη, 2)
    rmη = zeros(totseg, ar_order)
    rvη = zeros(totseg, ar_order, ar_order)
    for segnum in 1:totseg
        rmη[segnum, :], rvη[segnum, :, :] = reshape(mη, (ar_order,)), vη
    end
    priors_eta = rmη, rvη
    priors_tau = [τ for _ in 1:totseg]
    priors_eta[1], priors_eta[2], priors_tau
end

prior_to_priors (generic function with 1 method)

In [12]:
trmη_arr, trvη_arr, trτ_arr = prior_to_priors(trmη, trvη, trτ, tr_totseg)
bblmη_arr, bblvη_arr, bblτ_arr = prior_to_priors(bblmη, bblvη, bblτ, bbl_totseg);

In [13]:
function HA_algorithm(segments, priors_η, priors_τ, ar_1_order, ar_2_order, vmp_its)
    """Source seperation based on coupled AR model. Inference is performed in batch manner

       segments: segmented audio signal
       priors_η:   matrix of means and covariances of AR coefficients (see output formal of prior_to_priors
       priors_τ:   array of tupes contatining the prior of environmental noise precision
       ar_1_order: order of speech signal
       ar_2_order: order of environmental noise signal
       vmp_its:    number of variational iterations
    """
    n_sources = 2
    totseg = size(segments, 1)
    l      = size(segments, 2) # dimensionality of the buffer
    
    rmx = zeros(totseg, l)
    rvx = zeros(totseg, l)
    rmθ = zeros(totseg, ar_1_order)
    rvθ = zeros(totseg, ar_1_order, ar_1_order)
    rγ = fill(tuple(.0, .0), totseg)
    
    rmz = zeros(totseg, l)
    rvz = zeros(totseg, l)
    rmη = zeros(totseg, ar_2_order)
    rvη = zeros(totseg, ar_2_order, ar_2_order)
    rτ = fill(tuple(.0, .0), totseg)
    
    fe  = zeros(totseg, vmp_its)
    
    rmo = zeros(totseg, l)
    
    # agent proposes gains according to its beliefs
    ProgressMeter.@showprogress for segnum in 1:totseg
        prior_η                           = (priors_η[1][segnum, :], priors_η[2][segnum, :, :])
        prior_τ                           = priors_τ[segnum]
        γ, θ, zs, τ, η, xs, fe[segnum, :] = coupled_inference(segments[segnum, :], prior_η, prior_τ, ar_1_order, ar_2_order, vmp_its)
        mz, vz                            = mean.(zs), cov.(zs)
        mθ, vθ                            = mean(θ), cov(θ)
        rmz[segnum, :], rvz[segnum, :]    = first.(mz), first.(vz)
        rmθ[segnum, :], rvθ[segnum, :, :] = mθ, vθ
        rγ[segnum]                        = shape(γ), rate(γ)
        
        mx, vx                            = mean.(xs), cov.(xs)
        mη, vη                            = mean(η), cov(η)
        rmx[segnum, :], rvx[segnum, :]    = first.(mx), first.(vx)
        rmη[segnum, :], rvη[segnum, :, :] = mη, vη
        rτ[segnum]                        = shape(τ), rate(τ)
        
        # HA part
        speech = rmz[segnum, :]
        noise  = rmx[segnum, :]
        rmo[segnum, :] = speech .+ noise
    end
    rmz, rvz, rmθ, rvθ, rγ, rmx, rvx, rmη, rvη, rτ, fe, rmo
end

HA_algorithm (generic function with 1 method)

#### Obtain the outputs from HA

In [14]:
# return list of files from dir_name
function get_sounds_fn(dir_name)
    file_names = []
    for (root, dirs, files) in walkdir(dir_name)
        for file in files
            push!(file_names, joinpath(root, file)) # path to files
        end
    end
    file_names
end

get_sounds_fn (generic function with 1 method)

In [15]:
training_files = get_sounds_fn("sound/AIDA/training/");

In [16]:
# Check for how the data is organized
training_file = training_files[8]
println(training_file)
speech, fs = WAV.wavread(training_file)
speech_seg = get_frames(speech, fs)

sound/AIDA/training/sp08_babble_sn0.wav


380×80 Matrix{Float64}:
  0.0542314   0.0467544    0.00265511  …  -0.126011    -0.206244
  0.033723    0.0573443    0.0773339      -0.0227058   -0.0657979
  0.140538    0.106693     0.0360118       0.146092     0.0616169
  0.0662252  -0.018128    -0.032075       -0.0811182    0.0431227
  0.0480972   0.0445875    0.0498367      -0.0253609   -0.0185553
 -0.0292673  -0.0645466   -0.103793    …   0.14066      0.123875
 -0.0381481  -0.0107425    0.0104678       0.0936613    0.0746178
 -0.028077   -0.0843837   -0.0903348      -0.036256    -0.0307016
  0.0492569   0.0639973    0.0399182       0.050264    -0.0648824
  0.0670797   0.00317392   0.00521867      0.044496     0.0498978
 -0.101077   -0.0573138   -0.014069    …  -0.0697348   -0.0841395
  0.0554827   0.0452589   -0.0309763       0.156255     0.0937223
 -0.0675069   0.0357677    0.0458998      -0.156713    -0.0555437
  ⋮                                    ⋱               
  0.0116581  -0.00210578  -0.0115665      -0.0350963   -0.021149

At this stage we run our inference algorithm to seperate **z** and **x**. We write the output into *.jld* files.

You don't need to run this snippet if you haven't changed the default parameters of HA_algorithm and priors (just see *sound/AIDA/separated_jld/training/*)

In [312]:
import JLD
# r for recovered
for training_file in training_files
    println(training_file)
    speech, fs = WAV.wavread(training_file)
    speech_seg = get_frames(speech, fs)
    # choose priors
    priors_eta = occursin("babble", training_file) ? (bblmη_arr, bblvη_arr) : (trmη_arr, trvη_arr)
    priors_tau = occursin("babble", training_file) ? bblτ_arr : trτ_arr
    # make sure that 1D of priors and speech_seg are equal
    if size(priors_eta, 1) != size(speech_seg, 1)
        totseg = size(speech_seg, 1)
        priors_eta_m, priors_eta_v, priors_tau = prior_to_priors(priors_eta[1][1, :]', priors_eta[2][1, :, :], priors_tau[1], totseg)
        priors_eta = priors_eta_m, priors_eta_v
    end
    rmz, rvz, rmθ, rvθ, rγ, rmx, rvx, rmη, rvη, rτ, fe, rmo = HA_algorithm(speech_seg, priors_eta, priors_tau, 10, 2, 10);
    
    JLD.save("sound/AIDA/separated_jld/training/"*training_file[findfirst("sp", training_file)[1]:end][1:end-3]*"jld",
         "rmz", rmz, "rvz", rvz, "rmθ", rmθ, "rvθ", rvθ, "rγ", rγ, 
         "rmx", rmx, "rvx", rvx, "rmη", rmη, "rvη", rvη, "rτ", rτ,
         "fe", fe, "rmo", rmo, "filename", training_file,
         "audio", speech)
end

sound/AIDA/training/sp08_babble_sn0.wav


Progress: 100%|█████████████████████████████████████████| Time: 0:03:39


sound/AIDA/training/sp09_babble_sn0.wav


Progress: 100%|█████████████████████████████████████████| Time: 0:03:46


sound/AIDA/training/sp10_babble_sn0.wav


...

Progress: 100%|█████████████████████████████████████████| Time: 0:02:51


sound/AIDA/training/sp19_train_sn0.wav


Progress: 100%|█████████████████████████████████████████| Time: 0:03:20


sound/AIDA/training/sp20_train_sn0.wav


Progress: 100%|█████████████████████████████████████████| Time: 0:03:14


### Preference learning stage

#### Generate outputs of HA from JLD files

In [15]:
# obtain saved jld files containing inference result
training_jlds = get_sounds_fn("sound/AIDA/separated_jld/training/");

In [18]:
# initialize gains that agent randomly assigns to the outputs of HA
agent_gains = [[2.0, 1.0], [1.0, 0.0], [0.5, 0.5], [0.9, 0.3], [2.5, 1.0]];

In [ ]:
# this snippet generates new ha outputs
for training_jld in training_jlds
    # read file
    d = JLD.load(training_jld)
    filename = d["filename"]
    # extract speech
    rmz, rmx = d["rmz"], d["rmx"]
    # extract noise
    rz, rx = get_signal(rmz, fs), get_signal(rmx, fs)
    # pick weights 
    whgs = rand(agent_gains) # can be changed to a smarter function
    # create output
    ha_out = whgs[1] .* rz + whgs[2] .* rx
    # write wav file
    WAV.wavwrite(ha_out, fs, "sound/AIDA/preference_learning/ha_out_$(whgs[1])_$(whgs[2])_"*filename[findfirst("sp", filename)[1]:end])
end

#### Create pairs (gains, context) <-> appraisals

In [20]:
# Recorded gains and appraisals 
# If you want to generate new pairs, please procceed with listening
gains = [[0.5, 0.5], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [2.0, 1.0], [2.0, 1.0], [2.5, 1.0], [2.5, 1.0], [2.5, 1.0]]
appraisals = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]
contexts = [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0];

In [14]:
prl_files = get_sounds_fn("sound/AIDA/preference_learning/");

#### Listening

User gets to listen new audio samples with proposed gains. After each listening he/she evaluates the performance of HA output by binary feedback.

In [ ]:
# This snippet creates interactive loop where user can like/dislike hearing aid output
nnum = 8 # prefix for gains
appraisals = []
gains = []
contexts = []
for prl_file in prl_files
    
    if !occursin("wav", prl_file)
        continue
    end
    WAV.wavplay(prl_file)
    println("How's HA output 1, 0 ?")
    appraisal = readline()
    push!(appraisals, parse(Float64, appraisal))
    
    # extract gains routine
    pref_id = findfirst("out_", prl_file)[end]
    gains_str = prl_file[pref_id+1:pref_id+nnum-1]
    push!(gains, parse.(Float64, split(gains_str, "_")))
    
    # extract context routine
    push!(contexts, occursin("babble", prl_file) ? 1.0 : 0.0)
end

In [21]:
println(gains)
println(appraisals)
println(contexts)

[[0.5, 0.5], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [2.0, 1.0], [2.0, 1.0], [2.5, 1.0], [2.5, 1.0], [2.5, 1.0]]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]


In [31]:
# sampling based agent. 
include("agent/agent_sampling.jl");

In [22]:
# Returns paramaters of a neural netwrok that provides the highest log posterior.
function learning(gains, contexts, appraisals, N=1000; g_jitter=1e-4)
    # adding jitter to gains might be useful when dealing with small number of observations
    gains = [g .+ sqrt(g_jitter)*randn(length(g)) for g in gains]
    inputs = [vcat(g, c) for (g, c) in zip(gains, contexts)]
    ch = Turing.sample(prefernce_learning(hcat(inputs...), appraisals, 1.0), Turing.HMC(0.05, 10), N);
    # Extract all weight and bias parameters.
    theta = Turing.MCMCChains.group(ch, :nn_params).value;
    # Find the index that provided the highest log posterior in the chain.
    _, i = findmax(ch[:lp])
    i = i.I[1]
    θ = Float64.(theta[i, :])
end

learning (generic function with 2 methods)

In [32]:
nn_params = learning(gains, contexts, appraisals);

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


In [33]:
# Alternative (in case you don't want to use learning function)
gains₊ = [g .+ sqrt(1e-4)*randn(length(g)) for g in gains]
inputs = [vcat(g, c) for (g, c) in zip(gains, contexts)]
ch = Turing.sample(prefernce_learning(hcat(inputs...), appraisals, 1.0), Turing.HMC(0.05, 10), 1000);

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


In [ ]:
sumstats = Turing.summarize(ch, Turing.mean, Turing.std)
mθ, vθ   = sumstats.nt.mean, sumstats.nt.std;

In [27]:
theta = Turing.MCMCChains.group(ch, :nn_params).value;
# Find the index that provided the highest log posterior in the chain.
_, i = findmax(ch[:lp])
i = i.I[1]
θ = theta[i, :];

In [28]:
predictions = []
for n in 1:length(appraisals)
    push!(predictions, nn_forward(inputs[n], θ))
end

In [33]:
# Check for errors on training set
appraisals_ = [rand(Distributions.Bernoulli(p[1])) for p in predictions]
err = sum(abs.(appraisals .- appraisals_))

7.0

## Planning
Few things must be said about the planning stage. 
First of all, the idea of planning is a reverse problem to prefernce learning. 
Given the parameters of neural network, the goal prior for the appraisal (1.0) and an informative prior for the future context (we have an idea of how the environment evolves), we want to infer the most suitable gains.
The evolution of the context will be based on HMM model, where the observations are 

For illustration purposes, we will first run the inference algorithm to obtain **z** and **x**. Secondly, we run our agent that proposes gains.

In [72]:
# infer gains
function sample_gains(context, nn_params, N=1000)
    ch = Turing.sample(planning(nn_params, [1.0], context), Turing.HMC(0.01, 4), N);
    gains_context = Turing.MCMCChains.group(ch, :gs).value
    # Find the index that provided the highest log posterior in the chain.
    _, i = findmax(ch[:lp])
    i = i.I[1]
    Float64.(gains_context[i, :])
end

sample_gains (generic function with 2 methods)

In [34]:
test_files = get_sounds_fn("sound/AIDA/test/");

In [25]:
# obtain HA output for test set (data for acting)
using JLD
# r for recovered
for test_file in test_files
    println(test_file)
    speech, fs = WAV.wavread(test_file)
    speech_seg = get_frames(speech, fs)
    # choose priors
    priors_eta = occursin("babble", test_file) ? (bblmη_arr, bblvη_arr) : (trmη_arr, trvη_arr)
    priors_tau = occursin("babble", test_file) ? bblτ_arr : trτ_arr
    
    # make sure that 1D of priors and speech_seg are equal
    if size(priors_eta, 1) != size(speech_seg, 1)
        totseg = size(speech_seg, 1)
        priors_eta_m, priors_eta_v, priors_tau = prior_to_priors(priors_eta[1][1, :]', priors_eta[2][1, :, :], priors_tau[1], totseg)
        priors_eta = priors_eta_m, priors_eta_v
    end
    rmz, rvz, rmθ, rvθ, rγ, rmx, rvx, rmη, rvη, rτ, fe, rmo = HA_algorithm(speech_seg, priors_eta, priors_tau, 10, 2, 10);
    
    JLD.save("sound/AIDA/separated_jld/test/"*test_file[findfirst("sp", test_file)[1]:end][1:end-3]*"jld",
         "rmz", rmz, "rvz", rvz, "rmθ", rmθ, "rvθ", rvθ, "rγ", rγ, 
         "rmx", rmx, "rvx", rvx, "rmη", rmη, "rvη", rvη, "rτ", rτ,
         "fe", fe, "rmo", rmo, "filename", test_file,
         "audio", speech)
end

sound/AIDA/test/sp09_train_sn0.wav


Progress: 100%|█████████████████████████████████████████| Time: 0:04:12


sound/AIDA/test/sp10_train_sn0.wav


Progress: 100%|█████████████████████████████████████████| Time: 0:04:19


sound/AIDA/test/sp11_babble_sn0.wav


...

Progress: 100%|█████████████████████████████████████████| Time: 0:03:04


sound/AIDA/test/sp19_babble_sn0.wav


Progress: 100%|█████████████████████████████████████████| Time: 0:03:49


sound/AIDA/test/sp20_babble_sn0.wav


Progress: 100%|█████████████████████████████████████████| Time: 0:03:48


In [87]:
test_jlds = get_sounds_fn("sound/AIDA/separated_jld/test/");

In [89]:
# Here agent plans and learns
for test_jld in test_jlds
    d = JLD.load(test_jld)
    rmz, rmx = d["rmz"], d["rmx"]
    filename = d["filename"]
    context = occursin("babble", filename) ? 1.0 : 0.0
    rz, rx = get_signal(rmz, fs), get_signal(rmx, fs)
    gs = sample_gains(context, Float64.(θ))
    ha_out = gs[1] .* rz + gs[2] .* rx
    # TODO: we can learn here as well. If the user provides a feedback to each proposal, we can feed this information back to our learning function.
    WAV.wavwrite(ha_out, fs, "sound/AIDA/planning/ha_out_$(gs[1])_$(gs[2])_"*filename[findfirst("sp", filename)[1]:end])
end